<a href="https://colab.research.google.com/github/damianschmidt/PAMSI/blob/master/generate_text_token_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Import Tensorflow and other libs
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import re
import time

In [2]:
# Import SpaCy
!sudo pip install -U spacy
!sudo python -m spacy download pl_core_news_sm
import pkg_resources,imp
imp.reload(pkg_resources)
import spacy
nlp = spacy.load("pl_core_news_sm")


  Using cached https://github.com/explosion/spacy-models/releases/download/pl_core_news_sm-3.2.0/pl_core_news_sm-3.2.0-py3-none-any.whl (58.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
Wcale
nie
chcemy
zdobywać
kosmosu
,
chcemy
tylko
rozszerzyć
Ziemię
do
jego
granic
.


In [3]:
# Upload text file
from google.colab import files
uploaded = files.upload()

Saving quo_vadis.txt to quo_vadis.txt
Saving ogniem_i_mieczem_tom_pierwszy.txt to ogniem_i_mieczem_tom_pierwszy.txt
Saving potop_tom_trzeci.txt to potop_tom_trzeci.txt
Saving potop_tom_drugi.txt to potop_tom_drugi.txt
Saving pan_wolodyjowski.txt to pan_wolodyjowski (1).txt
Saving potop_tom_pierwszy.txt to potop_tom_pierwszy (1).txt
Saving lalka_tom_pierwszy.txt to lalka_tom_pierwszy (1).txt
Saving krzyzacy_tom_pierwszy.txt to krzyzacy_tom_pierwszy (1).txt


In [11]:
# Read text
text = ''
books_to_read = ['quo_vadis.txt',
                 'ogniem_i_mieczem_tom_pierwszy.txt',
                 'pan_wolodyjowski.txt',
                 'potop_tom_pierwszy.txt',
                 'potop_tom_drugi.txt',
                 'potop_tom_trzeci.txt',
                 'lalka_tom_pierwszy.txt',
                 'krzyzacy_tom_pierwszy.txt']
for book in books_to_read:
  text += open(f'/content/{book}', 'rb').read().decode(encoding='utf-8')

doc = nlp(text[:1000000])
# Length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')
print(f'Length of text: {len(doc)} tokens')


# Clean some characters
for char in ['–', '—', '−']:
    text = text.replace(char, '-')
for char in ['«', '»', '”', '„']:
    text = text.replace(char, '"')
text = text.replace('§', 'par. ')

# Remove headers, multiple lines, line breaks, page nr., etc.
text = re.sub(r'\n\n[0-9]+\n+([0-9]+\n)?', '\n[line]\n', text)
text = re.sub(r'\nSpis treści\n', '\n', text)
text = re.sub(r'[0-9]+\. posiedzenie .*\n+', '\n[line]\n', text)
text = re.sub(r'\[line\]\n(.*\n){2}\n+[A-Z].*\n', '\n[line]\n', text)
text = re.sub(r'-\n+([A-Z\[].*\n+)+', '', text)
text = re.sub(r'\n\n(.*\n)\[line\]', '\n[line]', text)
text = re.sub(r'\[line\]\n[A-Z].*\n\n', '', text)
text = re.sub(r'\[line\]\n', '\n', text)
text = re.sub(r'-\n+', '', text)
text = re.sub(r'…', '...', text)
text = re.sub(r'([^\.\)\?\!:\n])\n+', r'\1 ', text)
text = re.sub(r'\n{3,}', '\n\n', text)

# Take a look at the first 250 characters in text
print('chars:', text[:250])

# Take a look for first 20 tokens
doc_clear = nlp(text[:1000000])
tokens = [token for token in doc_clear[:20]]
print('tokens:', tokens)

# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

Length of text: 7123816 characters
Length of text: 194270 tokens
 Petroniusz obudził się zaledwie koło południa i jak zwykle, zmęczony bardzo. Poprzedniego dnia był na uczcie u Nerona, która przeciągnęła się do późna w noc. Od pew
 , Petroniusz, obudził, się]
108 unique characters


In [9]:
# Process text
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


# Create training examples and targets
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)


def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text


dataset = sequences.map(split_input_target)

# Create training batches
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000


dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)


In [10]:
# Build model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = self.embedding(inputs, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)

    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x


vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, '(batch_size, sequence_lenght, vocab_size)')

model.summary()

(64, 100, 109) (batch_size, sequence_lenght, vocab_size)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  27904     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  111725    
                                                                 
Total params: 4,077,933
Trainable params: 4,077,933
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Train model
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# Configure checkpoints (later save checkpoints on my drive)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

EPOCHS = 30
hisotry = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/30
1104/1104 [==============================] - 153s 135ms/step - loss: 1.9534
Epoch 2/30
1104/1104 [==============================] - 149s 134ms/step - loss: 1.4434
Epoch 3/30
1104/1104 [==============================] - 150s 134ms/step - loss: 1.3428
Epoch 4/30
1104/1104 [==============================] - 150s 135ms/step - loss: 1.2872
Epoch 5/30
1104/1104 [==============================] - 150s 134ms/step - loss: 1.2485
Epoch 6/30
1104/1104 [==============================] - 150s 135ms/step - loss: 1.2187
Epoch 7/30
1104/1104 [==============================] - 150s 135ms/step - loss: 1.1950
Epoch 8/30
1104/1104 [==============================] - 150s 135ms/step - loss: 1.1757
Epoch 9/30
1104/1104 [==============================] - 150s 134ms/step - loss: 1.1611
Epoch 10/30
1104/1104 [==============================] - 150s 134ms/step - loss: 1.1496
Epoch 11/30
1104/1104 [==============================] - 149s 134ms/step - loss: 1.1416
Epoch 12/30
1104/1104 [==================

In [14]:
# Generate text
class OneStepPrediction(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1):
    super().__init__(self)
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step_prediction(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

one_step_prediction = OneStepPrediction(model, chars_from_ids, ids_from_chars)

start_time = time.time()
states = None
next_char = tf.constant(['Do'])
result = [next_char]

for n in range(200):
  next_char, states = one_step_prediction.generate_one_step_prediction(next_char, states=states)
  result.append(next_char)

print(len(result))
result = tf.strings.join(result)
print(len(result[0].numpy().decode('UTF-8')))
end_time = time.time()

print(result[0].numpy().decode('UTF-8'))
print('\nRuntime:', end_time - start_time)


201
202
 - Żaden! - mruknąć wierzące oklaskiem leśnego dzikowy wyrostkie. A już pewien z list potrącić okropnym potomencam

Runtime: 1.803819179534912


In [ ]:
# Save generator
tf.saved_model.save(one_step_prediction, 'one_step_prediction')
one_step_prediction_reloaded = tf.saved_model.load('one_step_prediction')

states = None
next_char = tf.constant(['Winston'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_prediction_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result[0]).numpy().decode('UTF-8'))